In [1]:
from pathlib import Path
from scanpy import read_h5ad
import pandas as pd
import anndata

tracker = pd.read_excel('~/surveyNAS05/scratch/MERSCOPE_Human_Imaging_Tracker.xlsx', sheet_name='TissueMetadata')
h5ad_folders = ['/home/imaging_mfish/MERSCOPENAS02_data/human/merfish_output/202112101342_H1930001Cx46MTG0202007205_VMSC01001/processed/',
                '/home/imaging_mfish/MERSCOPENAS02_data/human/merfish_output/202112021529_H1930001Cx46MTG202007203_VMSC01001/processed/',
                '/home/imaging_mfish/MERSCOPENAS02_data/human/merfish_output/202112021532_H1930001Cx46MTG202007204_vmsc00401/processed/',
                '/home/imaging_mfish/MERSCOPENAS02_data/human/merfish_output/202201271619_H1930002Cx46MTG202007102_VMSC01001/processed/',
                '/home/imaging_mfish/MERSCOPENAS02_data/human/merfish_output/202201281113_H1930002Cx46MTG202007104_vmsc00401/processed/',
                '/home/imaging_mfish/MERSCOPENAS02_data/human/merfish_output/202201271624_H1930002Cx46MTG202007105_vmsc00401/processed/',
                '/home/imaging_mfish/MERSCOPENAS02_data/human/merfish_output/202201311715_H2133013Cx24MTG02007106_VMSC01001/processed/',
                '/home/imaging_mfish/MERSCOPENAS02_data/human/merfish_output/202201281456_H2133013Cx24MTG02007105_VMSC01001/processed/',
                '/home/imaging_mfish/surveyNAS05/scratch/vizgen_download/analyzed_data/HuBrain_C30_VS47_VZG167a_5hrPhoto_V3_JH_3-15-2022/processed/',
                '/home/imaging_mfish/surveyNAS05/scratch/vizgen_download/analyzed_data/HuBrain_C30_VS47_VZG167a_5hrPhoto_V12_JH_3-15-2022/processed/',
                '/home/imaging_mfish/surveyNAS05/scratch/vizgen_download/analyzed_data/HuBrain_C26_VS47_VZG167a_5hrbleach_V6_03-12-2022/processed/',
                '/home/imaging_mfish/surveyNAS05/scratch/vizgen_download/analyzed_data/HuBrain_C26_VS47_VZG167a_AIBS72hr_V3_03-12-2022/processed/',
                '/home/imaging_mfish/surveyNAS05/scratch/vizgen_download/analyzed_data/HuBrain_C26_VS47_VZG167a_AIBS72hrPhoto_V11_JH_3-15-2022/processed/',
                '/home/imaging_mfish/MERSCOPENAS04_data/human/atlas/merfish_output/202204141051_H2133016Cx26MTG0200720101_VMSC02501/processed/',
                '/home/imaging_mfish/MERSCOPENAS04_data/human/atlas/merfish_output/202204151539_H2133016Cx26MTG0200720102_VMSC02501/processed/',
                '/home/imaging_mfish/MERSCOPENAS04_data/human/atlas/merfish_output/202204181242_H2133016Cx26MTG0200720103_VMSC02501/processed/']

h5ad_folder_paths = [Path(x) for x in h5ad_folders]
h5ad_file_paths = [list(x.glob('*.h5ad'))[0] for x in h5ad_folder_paths]

In [2]:
vizgen_conversion_dict = {'HuBrain_C26_VS47_VZG167a_5hrbleach_V6_03-12-2022':'H21.33.015.Cx26.MTG.02.007.1.0', 
                          'HuBrain_C26_VS47_VZG167a_AIBS72hr_V3_03-12-2022': 'H21.33.015.Cx26.MTG.02.007.1.1',
                          'HuBrain_C26_VS47_VZG167a_AIBS72hrPhoto_V11_JH_3-15-2022': 'H21.33.015.Cx26.MTG.02.007.1.2',
                          'HuBrain_C30_VS47_VZG167a_5hrPhoto_V3_JH_3-15-2022': 'H21.33.019.Cx30.MTG.02.007.5.0',
                          'HuBrain_C30_VS47_VZG167a_5hrPhoto_V12_JH_3-15-2022': 'H21.33.019.Cx30.MTG.02.007.5.1',
                          'HuBrain_C30_VS47_VZG167a_AIBS72hrPhoto_V1_JH_3-15-2022': 'H21.33.019.Cx30.MTG.02.007.5.2',
                          'HuBrain_C26_VS47_VZG167a_5hrPhoto_V6_JH_3-15-2022': 'H21.33.015.Cx26.MTG.02.007.1'}

In [4]:
for file in h5ad_file_paths:
    adata = read_h5ad(file)
    fname = file.stem
    print(fname)
    if adata.uns['section'] == 'vizgen_run' or adata.uns['section'] =='in_':
        adata.uns['section'] = vizgen_conversion_dict[fname] 
    if len(adata.uns['section']) < 11:
        loose_section_start = fname.find('_')
        loose_section_end = fname.find('_', loose_section_start+1)
        lsec = fname[loose_section_start+1:loose_section_end]
        if 'MTG02' in lsec:
            section = lsec[0:3]+'.'+lsec[3:5]+'.'+lsec[5:8]+'.'+lsec[8:12]+'.'+lsec[12:15]+'.'+lsec[15:17]+'.'+lsec[17:19]+'.'+lsec[19:22]+'.'+lsec[23:]
        else:
            section = lsec[0:3]+'.'+lsec[3:5]+'.'+lsec[5:8]+'.'+lsec[8:12]+'.'+lsec[12:16]+'.'+lsec[16:18]+'.'+lsec[18:21]+'.'+lsec[22:]
        adata.uns['section'] = section
        print(section)
    #use section to get info from tracker
    aibs_id = adata.uns['section'][0:10]
    adata.obs['section'] = adata.uns['section']
    metadata = tracker[tracker['AIBS ID']==aibs_id]
    #add tracker info to uns    
    if not metadata.empty:
        mdata = {}
        for col, val in metadata.iteritems():
            try:
                mdata[str(col)] = str(val.values.tolist()[0])
            except IndexError:
                mdata[str(col)] = 'No Tracker info'
#             mdata['appended_section_id'] = '.'+str(counter)
            adata.uns['metadata'] = {aibs_id: mdata}
    #if no tracker info (merscopenas02 nonad) just add 'non-ad' and move on for now
    else:
        adata.uns['metadata'] = {aibs_id: {'Cognitive Status':'No Dementia', 'AIBS ID': aibs_id}}
    adata.write_h5ad(file)

202112101342_H1930001Cx46MTG0202007205_VMSC01001
202112021529_H1930001Cx46MTG202007203_VMSC01001
202112021532_H1930001Cx46MTG202007204_vmsc00401
202201271619_H1930002Cx46MTG202007102_VMSC01001
202201281113_H1930002Cx46MTG202007104_vmsc00401
202201271624_H1930002Cx46MTG202007105_vmsc00401
202201311715_H2133013Cx24MTG02007106_VMSC01001
202201281456_H2133013Cx24MTG02007105_VMSC01001
HuBrain_C30_VS47_VZG167a_5hrPhoto_V3_JH_3-15-2022
HuBrain_C30_VS47_VZG167a_5hrPhoto_V12_JH_3-15-2022
HuBrain_C26_VS47_VZG167a_5hrbleach_V6_03-12-2022
HuBrain_C26_VS47_VZG167a_AIBS72hr_V3_03-12-2022
HuBrain_C26_VS47_VZG167a_AIBS72hrPhoto_V11_JH_3-15-2022
202204141051_H2133016Cx26MTG0200720101_VMSC02501
202204151539_H2133016Cx26MTG0200720102_VMSC02501
202204181242_H2133016Cx26MTG0200720103_VMSC02501


In [77]:
#concatenate here
h5ad_raws = []
for file in h5ad_file_paths:
    x = read_h5ad(file)
    h5ad_raws.append(x)
combo_anndata = anndata.concat(h5ad_raws, uns_merge='first')

In [78]:
#adjust the cirro for the combined file
counter = 0
for ii, gb in combo_anndata.obs.groupby("section"): #ii = section name, gb = indexed chunk of dataframe
    spatial = combo_anndata.obsm['spatial'][combo_anndata.obs.section==ii,:]
    cirro_y = -(spatial[:,1] - np.mean(spatial[:,1]))
    cirro_x = (spatial[:,0] - min(spatial[:,0])) +(counter*10000)
    counter += 1
    coordinates_cirro = np.column_stack((cirro_x, cirro_y))
    combo_anndata.obsm['spatial_cirro'][np.where(combo_anndata.obs.section==ii)] = coordinates_cirro
combo_anndata.write_h5ad('/home/imaging_mfish/surveyNAS05/scratch/test_combo_050622.h5ad')